# Redução de Turnover dos Colaboradores

### Objetivo

   Este projeto de análise de dados visa utilizar um conjunto diversificado de __método estatísticos__ para investigar e responder ao questionamento levantado pelo departamento de Gestão de Pessoas: quais estratégias organizacionais ou fatores internos, se alterados, poderiam auxiliar na __diminuição da rotatividade de funcionários (Turnover)?__



### Técnicas aplicadas:

- Análise exploratória das variáveis com as bibliotecas YData-Profiling e SweetViz;
- Coeficiente de Person;
- Information Value (IV).

### Códigos desenvolvidos.

- Função para gerar os relatórios Ydata e SweetViz de forma automática;
- Classe com funções desenvolvidas para explorarmos a técnica __Information Value.__ 


## Importação das bibliotecas utilizadas

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import os
import aed
from IPython.display import display, HTML
import pandas_profiling 
import sweetviz as sv

## Funções criadas

In [3]:
def gera_relatorios_aed(df, target_feat, 
                        html_pp='base_aed_pp.html', 
                        html_sv='base_aed_sv.html'):
    '''
    '''
    # Gera relatório usando Pandas Profiling
    perfil_pp = df.profile_report()
    perfil_pp.to_file(output_file=html_pp)
    
    # Gera relatório usando SweetViz
    perfil_sv = sv.analyze(df, target_feat=target_feat)
    perfil_sv.show_html(html_sv)
    
    return perfil_pp, perfil_sv

In [4]:
class analise_iv: # Classe desenvolvida para calcularmos o information Value
        
    # função private
    def __get_tab_bivariada(self, var_escolhida):
     
        # Cria a contagem de Target_1 e Target_0
        df_aux = self.df.copy() 
        df_aux['target2'] = self.df[self.target]
        df2 = df_aux.pivot_table(values='target2',
                                 index=var_escolhida,
                                 columns=self.target,
                                 aggfunc='count')
        
        df2 = df2.rename(columns={0:'#Target_0',
                                  1:'#Target_1'})
        df2.fillna(0, inplace=True)

        # Cria as demais colunas da tabela bivariada
        df2['Total'] = (df2['#Target_0'] + df2['#Target_1'])
        df2['%Freq'] = (df2['Total'] / (df2['Total'].sum()) * 100).round(decimals=2)
        df2['%Target_1'] = (df2['#Target_1'] / (df2['#Target_1'].sum()) * 100).round(decimals=2)
        df2['%Target_0'] = (df2['#Target_0'] / (df2['#Target_0'].sum()) * 100).round(decimals=2)
        df2['%Target_0'] = df2['%Target_0'].apply(lambda x: 0.01 if x == 0 else x) #corrige problema do log indeterminado
        df2['%Taxa_de_Target_1'] = (df2['#Target_1'] / df2['Total'] * 100).round(decimals=2)
        df2['Odds'] = (df2['%Target_1'] / df2['%Target_0']).round(decimals=2)
        df2['Odds'] = df2.Odds.apply(lambda x: 0.01 if x == 0 else x) #corrige problema do log indeterminado
        df2['LN(Odds)'] = np.log(df2['Odds']).round(decimals=2)
        df2['IV'] = (((df2['%Target_1'] / 100 - df2['%Target_0'] / 100) * df2['LN(Odds)'])).round(decimals=2)
        df2['IV'] = np.where(df2['Odds'] == 0.01, 0 , df2['IV']) 

        df2 = df2.reset_index()
        df2['Variavel'] = var_escolhida
        df2 = df2.rename(columns={var_escolhida: 'Var_Range'})
        df2 = df2[['Variavel','Var_Range', '#Target_1','#Target_0', 'Total', '%Freq', '%Target_1', '%Target_0',
       '%Taxa_de_Target_1', 'Odds', 'LN(Odds)', 'IV']]
        
        # Guarda uma cópia da tabela no histórico
        self.df_tabs_iv = pd.concat([self.df_tabs_iv, df2], axis = 0)
        
        return df2
        
    def get_bivariada(self, var_escolhida='all_vars'):
        
        if var_escolhida == 'all_vars':
                       
            #vars = self.df.drop(self.target,axis = 1).columns
            vars = self.get_lista_iv().index
            for var in vars:
                tabela = self.df_tabs_iv[self.df_tabs_iv['Variavel'] == var]
                print('==> "{}" tem IV de {}'.format(var,self.df_tabs_iv[self.df_tabs_iv['Variavel'] == var]['IV'].sum().round(decimals=2)))
                # printa a tabela no Jupyter
                display(tabela)
            
            return
        
        else:
            print('==> "{}" tem IV de {}'.format(var_escolhida,self.df_tabs_iv[self.df_tabs_iv['Variavel'] == var_escolhida]['IV'].sum().round(decimals=2)))
            return self.df_tabs_iv[self.df_tabs_iv['Variavel'] == var_escolhida]
                   
            
    def get_lista_iv(self):
        
    
        # agrupa a lista de IV's em ordem descrescente
        lista = (self.df_tabs_iv.groupby('Variavel').agg({'IV':'sum'})).sort_values(by=['IV'],ascending=False)
            
        return lista
    
    

    def __init__(self, df, target, nbins=10):

        self.df = df.copy()
        self.target = target

        #lista de variaveis numericas
        df_num = self.df.loc[:,((self.df.dtypes == 'int32') | 
                                (self.df.dtypes == 'int64') | 
                                (self.df.dtypes == 'float64')
                               )
                            ]

        vars = df_num.drop(target,axis = 1).columns

        for var in vars:
            nome_var = 'fx_' + var 
            df_num[nome_var] = pd.qcut(df_num[var], 
                                       q=nbins, 
                                       precision=2,
                                       duplicates='drop')
            df_num = df_num.drop(var, axis = 1)
            df_num = df_num.rename(columns={nome_var: var})

        #lista de variaveis qualitativas
        df_str = self.df.loc[:,((self.df.dtypes == 'object') | 
                                (self.df.dtypes == 'category') |
                                (self.df.dtypes == 'bool'))]


        self.df = pd.concat([df_num,df_str],axis = 1)


         # inicializa tab historica
        self.df_tabs_iv = pd.DataFrame()

        vars = self.df.drop(self.target,axis = 1).columns
        for var in vars:
            self.__get_tab_bivariada(var);

        # remove tabs de iv duplicadas
        self.df_tabs_iv = self.df_tabs_iv.drop_duplicates(subset=['Variavel','Var_Range'], keep='last')

# 1. Análise Exploratória

In [5]:
df = pd.read_excel('Base_RH.xlsx',index_col=0)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1470 entries, 1 to 1470
Data columns (total 20 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   Funcionário_deixou_a_empresa  1470 non-null   object
 1   Idade                         1470 non-null   int64 
 2   Frequência de Viagens         1470 non-null   object
 3   Distância_do_trabalho         1470 non-null   int64 
 4   Formação                      1470 non-null   object
 5   E-Sat                         1470 non-null   object
 6   Gênero                        1470 non-null   object
 7   Estado_Civil                  1470 non-null   object
 8   Salário                       1470 non-null   int64 
 9   Qte_Empresas_Trabalhadas      1470 non-null   int64 
 10  Faz_hora_extras?              1470 non-null   object
 11  Perc_de_aumento               1470 non-null   int64 
 12  Qte_ações_da_empresa          1470 non-null   int64 
 13  Tempo_de_carreira 

<br><br>
Temos __8 variáveis qualitativas__, sendo __duas delas ordinais__.<br> E __12 variáveis quantitativas discreta.__
<br><br>


In [7]:
df.head()

,Funcionário_deixou_a_empresa,Idade,Frequência de Viagens,Distância_do_trabalho,Formação,E-Sat,Gênero,Estado_Civil,Salário,Qte_Empresas_Trabalhadas,Faz_hora_extras?,Perc_de_aumento,Qte_ações_da_empresa,Tempo_de_carreira,Horas_de_treinamento,Equilibrio_de_Vida,Tempo_de_empresa,Anos_no_mesmo_cargo,Anos_desde_a_ultima_promocao,Anos_com_o_mesmo_chefe
ID,,,,,,,,,,,,,,,,,,,,
1,Sim,41,Viaja raramente,1,Ensino Técnico,Médio,M,Solteiro,5993,8,Sim,11,0,8,0,Ruim,6,4,0,5
2,Não,49,Viaja frequentemente,8,Ensino Médio,Alto,H,Casado,5130,1,Não,23,1,10,3,Muito bom,10,7,1,7
3,Sim,37,Viaja raramente,2,Ensino Técnico,Muito Alto,H,Solteiro,2090,6,Sim,15,0,7,3,Muito bom,0,0,0,0
4,Não,33,Viaja frequentemente,3,Mestrado,Muito Alto,M,Casado,2909,1,Sim,11,0,8,3,Muito bom,8,7,3,0
5,Não,27,Viaja raramente,2,Ensino Médio,Baixo,H,Casado,3468,9,Não,12,1,6,3,Muito bom,2,2,2,2


In [8]:
df.duplicated().sum()

0

In [9]:
df.isnull().sum()

Funcionário_deixou_a_empresa    0
Idade                           0
Frequência de Viagens           0
Distância_do_trabalho           0
Formação                        0
E-Sat                           0
Gênero                          0
Estado_Civil                    0
Salário                         0
Qte_Empresas_Trabalhadas        0
Faz_hora_extras?                0
Perc_de_aumento                 0
Qte_ações_da_empresa            0
Tempo_de_carreira               0
Horas_de_treinamento            0
Equilibrio_de_Vida              0
Tempo_de_empresa                0
Anos_no_mesmo_cargo             0
Anos_desde_a_ultima_promocao    0
Anos_com_o_mesmo_chefe          0
dtype: int64

<br><br>
Não temos valores __nulos__ ou __duplicados__ na base.
<br><br>

# 1.1 Determinação do Turnover: 

In [10]:
df['Funcionário_deixou_a_empresa'].value_counts()/df['Funcionário_deixou_a_empresa'].value_counts().sum()*100

Não    83.877551
Sim    16.122449
Name: Funcionário_deixou_a_empresa, dtype: float64

<br><br>
A __taxa de turnover__ nessa empresa é de __`16,12%`__ 



É um valor considerado alto pois a saída de um colaborador implica custos financeiros e de tempo para a empresa, que precisa investir em processos de recrutamento e treinamento de novos funcionários, além de enfrentar uma possível queda na produtividade do setor afetado.<br>
Segundo a Qulture Rocks (https://www.qulture.rocks/blog/turnover-ideal), a taxa de turnover ideal é em torno de __`10 %`__.<br> A saída de funcionários em quantidade controlada, é importante para que haja o fortalecimento da cultura organizacional, o que demonstra que os profissionais desalinhados estão deixando o negócio, abrindo espaço para que novos talentos façam parte do time, que são responsáveis por oxigenar o ambiente empresarial, trazendo novas ideias e inovações.
<br><br>

# 1.2 Exploração da base com as bibliotecas YData Profile e SweetViz

### 1.2.1 Criando a coluna target para obter melhor performance da biblioteca SweetViz.

In [11]:
df['target'] = np.where(df['Funcionário_deixou_a_empresa']=='Sim',1,0)

In [12]:
df.head()

,Funcionário_deixou_a_empresa,Idade,Frequência de Viagens,Distância_do_trabalho,Formação,E-Sat,Gênero,Estado_Civil,Salário,Qte_Empresas_Trabalhadas,...,Perc_de_aumento,Qte_ações_da_empresa,Tempo_de_carreira,Horas_de_treinamento,Equilibrio_de_Vida,Tempo_de_empresa,Anos_no_mesmo_cargo,Anos_desde_a_ultima_promocao,Anos_com_o_mesmo_chefe,target
ID,,,,,,,,,,,,,,,,,,,,,
1,Sim,41,Viaja raramente,1,Ensino Técnico,Médio,M,Solteiro,5993,8,...,11,0,8,0,Ruim,6,4,0,5,1
2,Não,49,Viaja frequentemente,8,Ensino Médio,Alto,H,Casado,5130,1,...,23,1,10,3,Muito bom,10,7,1,7,0
3,Sim,37,Viaja raramente,2,Ensino Técnico,Muito Alto,H,Solteiro,2090,6,...,15,0,7,3,Muito bom,0,0,0,0,1
4,Não,33,Viaja frequentemente,3,Mestrado,Muito Alto,M,Casado,2909,1,...,11,0,8,3,Muito bom,8,7,3,0,0
5,Não,27,Viaja raramente,2,Ensino Médio,Baixo,H,Casado,3468,9,...,12,1,6,3,Muito bom,2,2,2,2,0


### 1.2.2 Criação dos relatórios com função desenvolvida na célula 2

In [12]:
perfil_pp, perfil_sv = gera_relatorios_aed(df=df,target_feat='target')

:TARGET::                          |▉                    | [  5%]   00:00  -> (00:00 left)

:FEATURES DONE:                    |█████████████████████| [100%]   00:04  -> (00:00 left)
:PAIRWISE DONE:                    |█████████████████████| [100%]   00:00  -> (00:00 left)


Creating Associations graph... DONE!
Report base_aed_sv.html was generated! NOTEBOOK/COLAB USERS: no browser will pop up, the report is saved in your notebook/colab files.


# 2. Information Value

É uma técnica estatística responsável por fornecer o __"poder de separação"__ que uma variável de duas ou mais categorias possui sobre outra variável de duas categorias (variável binária), ou seja, é utilizada para quantificar a importância de uma variável na previsão de um resultado.

Portanto vamos verificar quais das variáveis disponíveis tem maior poder de separar a categoria de funcionários que deixaram a empresa.

### Instânciando a classe criada na célula 2

In [13]:
df_iv = analise_iv(df.drop('Funcionário_deixou_a_empresa', axis =1),'target', nbins = 5)

C:\Users\Administrador\AppData\Local\Temp\ipykernel_21092\1257277310.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_num[nome_var] = pd.qcut(df_num[var],


In [14]:
df_iv.get_lista_iv()

,IV
Variavel,
Faz_hora_extras?,0.40
Tempo_de_empresa,0.31
Salário,0.31
Anos_no_mesmo_cargo,0.26
Tempo_de_carreira,0.26
Idade,0.26
Anos_com_o_mesmo_chefe,0.25
Estado_Civil,0.22
Frequência de Viagens,0.13


Para avaliar o __poder de separação__, na prática é utilizado a referência do benchmark, demonstrado na tabela abaixo:



|IV Total |Poder de Separação|
|----------|----------|
| < 0,02  | Muito Fraco |
| 0,02 a 0,1  | Fraco  |
| 0,1 a 0,3  | Médio  |
| 0,3 a 0,5 | Forte |
| > 0,5 | Atenção |

Ao aplicarmos a técnica estatística para determinação do IV, vemos que os principais fatores que impactam na alta taxa de turnover são __horas extras, tempo de empresa e salário__, e com menos intensidade, mas com importância ainda relevante são os __anos no mesmo cargo, tempo de carreira e idade.__<br><br>
Portanto, vamos explorar com maior detalhe cada variável mencionada.

## 2.1 Variável horas Extras

In [16]:
df_iv.get_bivariada(var_escolhida='Faz_hora_extras?')

==> "Faz_hora_extras?" tem IV de 0.4


target,Variavel,Var_Range,#Target_1,#Target_0,Total,%Freq,%Target_1,%Target_0,%Taxa_de_Target_1,Odds,LN(Odds),IV
0,Faz_hora_extras?,Não,110,944,1054,71.7,46.41,76.56,10.44,0.61,-0.49,0.15
1,Faz_hora_extras?,Sim,127,289,416,28.3,53.59,23.44,30.53,2.29,0.83,0.25


 <img src="Imagens/faz_hora_extra.png">

Ao detalharmos a variável com as informações dos colaboradores que fazem  horas extras, percebemos que a __taxa de turnover é de `31%`__ para quem __realiza horas extras__ frente a __`10%` de turnover__ para quem __não a faz__.<br>
Pelo cálculo da Odds na determinação do IV, quem __faz horas extras tem 2,29__ vezes mais chances de sair da empresa.

## 2.2 Variável Tempo de Empresa

In [17]:
df_iv.get_bivariada(var_escolhida='Tempo_de_empresa')

==> "Tempo_de_empresa" tem IV de 0.31


target,Variavel,Var_Range,#Target_1,#Target_0,Total,%Freq,%Target_1,%Target_0,%Taxa_de_Target_1,Odds,LN(Odds),IV
0,Tempo_de_empresa,"(-0.01, 2.0]",102,240,342,23.27,43.04,19.46,29.82,2.21,0.79,0.19
1,Tempo_de_empresa,"(2.0, 5.0]",60,374,434,29.52,25.32,30.33,13.82,0.83,-0.19,0.01
2,Tempo_de_empresa,"(5.0, 7.0]",20,146,166,11.29,8.44,11.84,12.05,0.71,-0.34,0.01
3,Tempo_de_empresa,"(7.0, 10.0]",35,247,282,19.18,14.77,20.03,12.41,0.74,-0.30,0.02
4,Tempo_de_empresa,"(10.0, 40.0]",20,226,246,16.73,8.44,18.33,8.13,0.46,-0.78,0.08


<img src='Imagens/Tempo_de_empresa2.png'>

In [28]:
#Quantidade de colaboradores por tempo de empresa
counts, bins, patches = plt.hist(df['Tempo_de_empresa'], bins=40)
print("Funcionários com até 2 anos de empresa, representam {:.2f}% do total da base analisada ".format(((counts[1]+counts[2])/counts.sum())*100))

Funcionários com até 2 anos de empresa, representam 20.27% do total da base analisada 


<br><br><br>
No gráfico é possível verificar a __redução da taxa de turnover__ a medida que o colaborador __ganha experiência dentro da empresa__. <br>
Na tabela de IV, ao verificarmos os valores das ODDS (probabilidade de sair da empresa / probabilidade de não sair), vemos que __colaboradores com até 2 anos de empresa (20% dos colaboradores), tem __`2,21`__ vezes chances de sair da empresa__, e que ao __passar esses 2 anos__, as chances do funcionário sair da empresa __é reduzida consideralvelmente__. <br>Pode-se concluir que o desenvolvimento de políticas para retenção de talentos é essencial para reduzir a taxa de turnover.

## 2.3 Variável Salário

In [30]:
df_iv.get_bivariada(var_escolhida='Salário')

==> "Salário" tem IV de 0.31


target,Variavel,Var_Range,#Target_1,#Target_0,Total,%Freq,%Target_1,%Target_0,%Taxa_de_Target_1,Odds,LN(Odds),IV
0,Salário,"(1008.99, 2695.8]",92,202,294,20.0,38.82,16.38,31.29,2.37,0.86,0.19
1,Salário,"(2695.8, 4228.8]",50,244,294,20.0,21.10,19.79,17.01,1.07,0.07,0.00
2,Salário,"(4228.8, 5743.4]",31,263,294,20.0,13.08,21.33,10.54,0.61,-0.49,0.04
3,Salário,"(5743.4, 9860.0]",37,257,294,20.0,15.61,20.84,12.59,0.75,-0.29,0.02
4,Salário,"(9860.0, 19999.0]",27,267,294,20.0,11.39,21.65,9.18,0.53,-0.63,0.06


 <img src="Imagens/Salário.png">

De forma geral, percebemos a tendência da redução da taxa de turnover, a medida que o salário aumento. Pela tabela IV, identificamos que quem ganha __até R$2695,00__ tem `2,37` vezes mais chances de sair da empresa, e que para os salários __acima de 4228,00__, as chances são consideravelmente reduzidas. 

Ocorre uma situação peculiar quando o colaborador está na faixa de salário por volta de __R$10000,00__. Nessa faixa de valor, ocorre o rompimento da tendência de redução de demissão em função do salário, registrando taxas de turnover próximas de `20%`. 
Umas hipóteses pode ser a sobrecarga de responsabilidades acumulada para as funções quais esses patamares de salários estão associadas.
Esse rompimento de tendência pode ter inúmeros fatores como causa, sendo necessário a validação de hipóteses junto ao especialista de negócios da empresa em questão.

## Variável Anos no mesmo cargo

In [32]:
df_iv.get_bivariada(var_escolhida='Anos_no_mesmo_cargo')

==> "Anos_no_mesmo_cargo" tem IV de 0.26


target,Variavel,Var_Range,#Target_1,#Target_0,Total,%Freq,%Target_1,%Target_0,%Taxa_de_Target_1,Odds,LN(Odds),IV
0,Anos_no_mesmo_cargo,"(-0.01, 1.0]",84,217,301,20.48,35.44,17.60,27.91,2.01,0.70,0.12
1,Anos_no_mesmo_cargo,"(1.0, 2.0]",68,304,372,25.31,28.69,24.66,18.28,1.16,0.15,0.01
2,Anos_no_mesmo_cargo,"(2.0, 4.0]",31,208,239,16.26,13.08,16.87,12.97,0.78,-0.25,0.01
3,Anos_no_mesmo_cargo,"(4.0, 7.0]",34,261,295,20.07,14.35,21.17,11.53,0.68,-0.39,0.03
4,Anos_no_mesmo_cargo,"(7.0, 18.0]",20,243,263,17.89,8.44,19.71,7.60,0.43,-0.84,0.09


Percebemos que quem tem até __um ano no cargo__, tem `2,01` mais chances de sair da empresa, o que reforça a necessidade do estabelecimento de política de retenção de talentos.

In [35]:
df_iv.get_bivariada(var_escolhida='Tempo_de_carreira')

==> "Tempo_de_carreira" tem IV de 0.26


target,Variavel,Var_Range,#Target_1,#Target_0,Total,%Freq,%Target_1,%Target_0,%Taxa_de_Target_1,Odds,LN(Odds),IV
0,Tempo_de_carreira,"(-0.01, 5.0]",91,225,316,21.50,38.40,18.25,28.80,2.10,0.74,0.15
1,Tempo_de_carreira,"(5.0, 8.0]",56,253,309,21.02,23.63,20.52,18.12,1.15,0.14,0.00
2,Tempo_de_carreira,"(8.0, 10.0]",35,263,298,20.27,14.77,21.33,11.74,0.69,-0.37,0.02
3,Tempo_de_carreira,"(10.0, 17.0]",30,231,261,17.76,12.66,18.73,11.49,0.68,-0.39,0.02
4,Tempo_de_carreira,"(17.0, 40.0]",25,261,286,19.46,10.55,21.17,8.74,0.50,-0.69,0.07


Colaboradores em __início de carreiro (até 5 anos)__, apresentam muita mais __chances de saída__. 
É válido a consideração da __contratação de funcionários experiente__. Com essa tomada de decisão estratégica, as __chances de redução da taxa de turnover__ dessa empresa são grandes.

In [37]:
df_iv.get_bivariada(var_escolhida='Idade')

==> "Idade" tem IV de 0.26


target,Variavel,Var_Range,#Target_1,#Target_0,Total,%Freq,%Target_1,%Target_0,%Taxa_de_Target_1,Odds,LN(Odds),IV
0,Idade,"(17.99, 29.0]",91,235,326,22.18,38.40,19.06,27.91,2.01,0.70,0.14
1,Idade,"(29.0, 34.0]",59,266,325,22.11,24.89,21.57,18.15,1.15,0.14,0.00
2,Idade,"(34.0, 38.0]",24,231,255,17.35,10.13,18.73,9.41,0.54,-0.62,0.05
3,Idade,"(38.0, 45.0]",29,262,291,19.80,12.24,21.25,9.97,0.58,-0.54,0.05
4,Idade,"(45.0, 60.0]",34,239,273,18.57,14.35,19.38,12.45,0.74,-0.30,0.02


O fator idade está altamente relacionado com o tempo de carreira, nesse sentido ambos apresentaram valores muito similares de importancia, determinado pelo IV.<br><br><br>

# Conclusão



A análise realizada neste estudo demonstrou que a taxa de Turnover da empresa, atualmente em __`16,12%`__, é significativamente superior ao ideal sugerido pela literatura, de cerca de 10%. Este excesso de rotatividade implica custos diretos e indiretos à organização, tais como processos de recrutamento, treinamento e impactos na produtividade. Diante dos fatores identificados, sugerimos uma série de políticas corporativas e estratégias para mitigar esse impacto causado pela saída de funcionário acima do limite ótimo.

As principais variáveis que influenciam a métrica de turnover, estão associadas a __horas extras__, __tempo de empresa__ e __salário__. <br>
Colaboradores que realizam horas extras apresentam uma taxa de turnover alarmante de `31%`, comparado aos `10%` de quem não realiza. Isso indica uma clara necessidade de revisão das práticas de gestão de carga de trabalho e controle de horas extras, a fim de evitar a sobrecarga dos colaboradores e promover um equilíbrio mais saudável entre a vida profissional e pessoal. <br> 
Uma política corporativa que incentiva o controle de horas extras, o desenvolvimento de programas de bem-estar e o incentivo ao uso de folgas e férias pode reduzir significativamente a rotatividade.<br><br>

O __tempo de empresa__ também se mostrou um fator crucial, com funcionários de até dois anos de casa apresentando `2,21` vezes mais chances de __sair da empresa__. Portanto, políticas de prevenção para novos colaboradores são essenciais. Programas de onboarding robustos e mentorias estruturadas, que oferecem suporte e feedback contínuo, podem ajudar a integrar novos funcionários de maneira mais eficaz, aumentando seu engajamento e, consequentemente, sua permanência. <br><br>

O __salário__ também declarado é um fator relevante, especialmente para colaboradores que ganham até `2.695,00`, com uma probabilidade __2,37 vezes maior de sair da empresa__. À medida que o salário aumenta, a taxa de rotatividade diminui, __exceto para a faixa de R$10.000,00__, onde observamos uma ruptura na tendência de redução de demissões. Isso pode ser reflexo de uma sobrecarga de responsabilidades para cargas mais altas. Assim, é importante considerar uma política de revisão salarial, aliada à redistribuição de responsabilidades, para evitar a saída de colaboradores valiosos. Além disso, a criação de planos de carreira com progressos salariais claros e incentivos por desempenho pode ser uma ferramenta poderosa para reter talentos.<br><br>

Colaboradores com __menos de um ano no cargo__ também apresentam maior propensão ao turnover, com uma probabilidade `2,01` vezes maior de deixar a empresa. Isso reforça a necessidade de políticas de concentração focadas no desenvolvimento profissional logo no início da trajetória do funcionário dentro da organização. A implementação de programas de desenvolvimento de carreira e feedback constante são ações fundamentais para mitigar essa saída precoce.

O fator __idade__, que se correlaciona fortemente com o __tempo de carreira__, também aponta que funcionários em __início de carreira__ (até cinco anos) têm __maior tendência de saída__. A contratação de profissionais mais experientes pode ser uma estratégia eficiente para diminuir a rotatividade, já que esses colaboradores tendem a se estabilizar na empresa com mais rapidez e a contribuir de forma mais sólida para a operação da empresa.

Sugestões de Políticas Corporativas:

* Controle de Horas Extras;
* Onboarding e Integração;
* Revisão Salarial e Carreira;
* Redistribuição de Responsabilidades;
* Retenção de Talentos.

Com base nessas políticas e estratégias, a empresa poderá não apenas reduzir sua taxa de Turnover, mas também criar um ambiente organizacional mais saudável, sustentável e alinhado aos seus objetivos de longo prazo, promovendo retenção de talentos e aumento dos resultados.